#Импорт библиотек

In [69]:
import requests
import pandas as pd
import numpy as np
import copy

#Сбор данных

Напишем функцию для получения списка специальностей c источником финансирования - finance ('Бюджетная основа', 'Полное возмещение затрат').

In [141]:
def get_speciality(url, finance):
    user_agent = {'user-agent': 'Mozilla/5.0 (X11; Linux aarch64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.188 Safari/537.36 CrKey/1.54.250320'}
    data = {
        'admissionCampaignType' : "Прием на обучение на бакалавриат/специалитет",
        'financingSource' : finance,
        'studyForm' : "Очная"
    }

    raw = requests.post(url=url, headers = user_agent, params = data).json()
    return raw['data']

Напишем функцию, которая будет собирать данные о поступающих на определенное направление - department.

In [142]:
def get_data(url, department, finance, orig):
    # Словарь ключей для получения кол-во мест
    place = {'Имеющие особое право':    'SpecialQuotaCount',
             'На общих основаниях':     'BudgetQuotaCount',
             'Отдельная квота':         'SeparateQuotaCount',
             'Целевой прием':           'TargetQuotaCount',
             'Полное возмещение затрат':'ExtraBudgetQuotaCount'}

    user_agent = {'user-agent': 'Mozilla/5.0 (X11; Linux aarch64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.188 Safari/537.36 CrKey/1.54.250320'}
    data = {
        'admissionCampaignType' : "Прием на обучение на бакалавриат/специалитет",
        'financingSource' : finance,
        'studyForm' : "Очная",
        'trainingDirection' : department,
        'sortDirection' : "sum",
    }

    raw = requests.post(url=url, headers = user_agent, params = data).json()

    for student in raw.get("data"):
        if student['Code'] == '':
            continue

        if orig == 'Y' and student['AtestOrig'] == 'N':
          continue

        category = student['Category']

        data_students['СНИЛС'].append(student['Code'])
        data_students['Специальность'].append(department)
        data_students['Приоритет'].append(int(student["SelectedPriority"]))
        data_students['БВИ'].append(student["NoExams"] == 'Y')
        data_students['Категория'].append(category)

        try:
            data_students['Балл'].append(int(student['SumScore']))
        except:
            data_students['Балл'].append(np.nan)

        if category not in count_place:
            count_place[category] = {}

        if department not in count_place[category]:
            count_place[category][department] = int(student[place[category]])


In [143]:
# Словарь для хранения информации о кол-во мест
count_place = {}
# Будущий DataFrame для хранения информации об абитуриентах
data_students = {'СНИЛС': [], 'Специальность': [], 'Балл': [], 'Приоритет': [], 'БВИ': [], 'Категория': []}

# Выберите одну из категорий источника финансирования: 'Бюджетная основа', 'Полное возмещение затрат'
finance = input()
# Выберите категорию аттестата: 'N' - не учитывая наличие оригинала, 'Y' - только оригинал документа
orig = input()

Бюджетная основа
N


In [144]:
url_speciality = "https://www.dvfu.ru/bitrix/services/main/ajax.php?mode=class&c=dvfu%3Aadmission.spd.new&action=getTrainingDirectionList"
fefu_speciality = get_speciality(url_speciality, finance)

In [145]:
fefu_speciality[:3]

['01.03.02 Прикладная математика и информатика',
 '01.03.04 Прикладная математика (Математические и цифровые методы в экономике и аналитике)',
 '02.03.01 Математика и компьютерные науки (Сквозные цифровые технологии)']

Формирование pandasDataFrame по всем специальностям.

In [146]:
url_data ='https://www.dvfu.ru/bitrix/services/main/ajax.php?mode=class&c=dvfu%3Aadmission.spd.new&action=getStudents'

for speciality in fefu_speciality:
    get_data(url_data, speciality, finance, orig)

In [147]:
data_students = pd.DataFrame(data_students)
data_students.shape

(42030, 6)

Удалим дубликаты.

In [148]:
data_students = data_students.drop_duplicates()
data_students.shape

(42021, 6)

#Алгоритм

Так как count_place - это словарь словарей, для того чтобы он не изменялся по ходу выполнения программы, сделаем глубокое копирование.

In [149]:
copy_place = copy.deepcopy(count_place)
df_students = data_students.copy()

Создадим пустой DataFrame, который будет хранить информацию о поступивших абитуриентах.

In [150]:
students_received = pd.DataFrame({'СНИЛС': [], 'Специальность': [], 'Балл': [], 'Приоритет': [], 'БВИ': [], 'Категория': []})

Напишем функцию, которая будет считать кол-во оставшихся мест на все специальности в определенной категории мест.

In [151]:
def get_count_place(place, category):
    return sum([place[category][i] for i in place[category]])

Посмотрим на общее кол-во мест в категории: На общих основаниях.

In [152]:
get_count_place(copy_place, 'На общих основаниях')

1799

Напишем функцию, которая будет формировать DataFrame с поступившими абитуриентами по определенной категории (category).

In [153]:
def incoming_students(category):
    global students_received, df_students

    not_students = 0
    while not_students != len(copy_place[category]): # Выполняем цикл до тех пор, пока в
        not_students = 0                              # не останется абитуриентов на все специальности

        for speciality in copy_place[category]:
            # Проверим остались ли места на данную специальность
            if copy_place[category][speciality] == 0:
                not_students += 1
                continue

            # Формируем DataFrame по специальности и сортируем в порядке убывания
            speciality_data = df_students[(df_students['Специальность'] == speciality) & (df_students['Категория'] == category)]\
                              .sort_values(by=['БВИ', 'Балл'], ascending=( False , False ))

            # Делаем выборку из первых абитуриентов
            speciality_data = speciality_data.iloc[:copy_place[category][speciality]]
            # Проверяем не пустой ли speciality_data
            if len(speciality_data) == 0:
                not_students += 1
                continue

            # Перебираем проходящих на данную специальность абитуриентов
            for i, student in speciality_data.iterrows():
                # Проверяем есть ли уже абитуриент в DataFrame, поступивших абитуриентов
                if student['СНИЛС'] in students_received['СНИЛС'].to_list():
                    # Проверяем меньше ли новый приоритет
                    if student['Приоритет'] < students_received[students_received['СНИЛС'] == student['СНИЛС']]['Приоритет'].values[0]:

                        # Так как значение нового приоритета меньше, убираем абитуриента со старой специальности и добавляем в новую
                        index = students_received[students_received['СНИЛС'] == student['СНИЛС']].index
                        copy_place[students_received.loc[index, 'Категория'].values[0]][students_received.loc[index, 'Специальность'].values[0]] += 1

                        # Удаление информацию об абитуриенте
                        students_received.drop(labels=[index[0]], inplace=True)

                    # Если значение исходного приоритета меньше нового, пропускаем абитуриента
                    else:
                      df_students.drop(labels=[i], inplace=True)
                      continue

                # Добаляем информацию о новом, поступившем абитуриенте
                item = pd.DataFrame([student])
                item["БВИ"] = item["БВИ"].astype("boolean")
                students_received = pd.concat([students_received, item])

                # Обновляем кол-во мест
                copy_place[student['Категория']][student['Специальность']] -= 1

                # Удаляем из общих данных уже поступивших абитуриентов
                df_students.drop(labels=[i], inplace=True)


Для бюджетной основы сначала определяем поступивших абитуриентов на общих основаниях, затем на квотные места, оставшиеся места переходят в места 'На общих основаниях', дополняем списки 'На общих основаниях'.\
Для платной основы всё легче, так как присутствуют только места 'На общих основаниях'.

In [154]:
def get_students_received():

    # Определяем поступивших абитуриентов на общих основаниях
    incoming_students('На общих основаниях')

    if finance == 'Бюджетная основа':
        kvota = ['Имеющие особое право', 'Отдельная квота', 'Целевой прием']

        for category in kvota:
            incoming_students(category)

        # Добавляем оставшиеся места в общие основания
        for category in kvota:
          for speciality in copy_place[category]:
              copy_place['На общих основаниях'][speciality] += copy_place[category][speciality]
              copy_place[category][speciality] = 0

        # Дополняем поступивших абитуриентов на общих основаниях
        incoming_students('На общих основаниях')


In [155]:
get_students_received()

Проверим размерности данных и кол-во уникальных СНИЛСов (для того чтобы убедится, что один абитуриент не попал в наш DataFrame несколько раз).


In [156]:
df_students.shape, students_received.shape

((35431, 6), (2873, 6))

In [157]:
students_received['СНИЛС'].nunique()

2873

Тут можно проверить на какую специальность проходит абитуриент.

In [90]:
snils = input('Введите снилс в формате xxx-xxx-xxx xx или уникальный код: ')
students_received[students_received['СНИЛС'] == snils]

Введите снилс в формате xxx-xxx-xxx xx или уникальный код: 1


,СНИЛС,Специальность,Балл,Приоритет,БВИ,Категория


Вычислим проходные баллы на специальности.

In [158]:
passing_score = students_received[students_received['БВИ'] == False]\
                .groupby(['Специальность', 'Категория'], as_index=False)\
                .agg({'Балл': min}).sort_values(by='Категория')

In [161]:
passing_score.head(5)

,Специальность,Категория,Балл
0,01.03.02 Прикладная математика и информатика,Имеющие особое право,168.0
154,39.03.01 Социология,Имеющие особое право,180.0
151,38.05.01 Экономическая безопасность (Экономико...,Имеющие особое право,188.0
146,38.03.05 Бизнес-информатика (Бизнес-аналитика),Имеющие особое право,183.0
143,38.03.02 Менеджмент,Имеющие особое право,214.0


#Сохранение данных

Сохраняем собранные данные о всех абитуриентах в Excel таблицу.

In [92]:
data_students.to_excel('students.xlsx')

Сохраним данные о поступивших абитуриентах в Excel таблицу.

In [160]:
writer = pd.ExcelWriter('passing_score_final.xlsx', engine='xlsxwriter')

for category in passing_score['Категория'].unique():
    passing_score[passing_score['Категория'] == category].to_excel(writer, sheet_name=category)

writer.close()